In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import pickle
import pvlib
import pandas as pd
import numpy as np
from datetime import datetime

In [52]:
full_pv = pickle.load(open("./raw/generation.pickle", "rb"))

In [53]:
full_pv = pd.DataFrame({"Datetime":full_pv.times, "AC": full_pv.ac})

In [54]:
full_pv

,Datetime,AC
time,,
2006-01-01 00:10:00,2006-01-01 00:10:00,-3.03
2006-01-01 01:10:00,2006-01-01 01:10:00,-3.03
2006-01-01 02:10:00,2006-01-01 02:10:00,-3.03
2006-01-01 03:10:00,2006-01-01 03:10:00,-3.03
2006-01-01 04:10:00,2006-01-01 04:10:00,-3.03
...,...,...
2010-12-31 19:10:00,2010-12-31 19:10:00,-3.03
2010-12-31 20:10:00,2010-12-31 20:10:00,-3.03
2010-12-31 21:10:00,2010-12-31 21:10:00,-3.03


In [55]:
rooftop_solar_array = full_pv[['Datetime', 'AC']].copy()
rooftop_solar_array.loc[:,'Datetime'] = pd.to_datetime(rooftop_solar_array.loc[:,'Datetime'])
rooftop_solar_array.set_index('Datetime', inplace=True)

rooftop_solar_array.rename(columns={"AC": "energy"}, inplace=True)

rooftop_solar_array = rooftop_solar_array.loc[:, ['energy']] / 10 # W to kW  and 100fold the generation 
rooftop_solar_array = rooftop_solar_array.resample('min').interpolate(method='linear')

In [56]:
print("Max energy: ", np.max(rooftop_solar_array["energy"].values))
print("Min energy: ", np.min(rooftop_solar_array["energy"].values))


Max energy:  44.00727161894718
Min energy:  -0.42697428328038


In [57]:
rooftop_solar_array_episodes = rooftop_solar_array.groupby(pd.Grouper(freq='W'))

In [58]:
rooftop_solar_array_episodes.describe()

energy                                                        \
              count      mean       std       min    25%    50%       75%   
Datetime                                                                    
2006-01-01   1430.0  0.173051  1.186114 -0.303000 -0.303 -0.303 -0.301974   
2006-01-08  10080.0  0.320700  1.417695 -0.303000 -0.303 -0.303  0.083451   
2006-01-15  10080.0  0.193985  1.170009 -0.303000 -0.303 -0.303 -0.093494   
2006-01-22  10080.0  0.665450  1.922868 -0.303000 -0.303 -0.303  0.498948   
2006-01-29  10080.0  0.539333  1.879291 -0.335367 -0.303 -0.303  0.380404   
...             ...       ...       ...       ...    ...    ...       ...   
2010-12-05  10080.0  0.640491  1.859854 -0.401068 -0.303 -0.303  0.158800   
2010-12-12  10080.0  0.331595  1.507370 -0.343760 -0.303 -0.303 -0.124144   
2010-12-19  10080.0  0.231661  1.219299 -0.303000 -0.303 -0.303 -0.210488   
2010-12-26  10080.0  0.485809  1.608738 -0.353163 -0.303 -0.303  0.061861   
2011-01-02   7151.0  0.493839  1.688905 -0.349843 -0.303 -0.303 -0.024307   

                      
                 max  
Datetime              
2006-01-01  5.163929  
2006-01-08  6.817939  
2006-01-15  6.502937  
2006-01-22  7.988411  
2006-01-29  9.481257  
...              ...  
2010-12-05  7.398300  
2010-12-12  6.982737  
2010-12-19  6.514273  
2010-12-26  6.321891  
2011-01-02  6.717487  

[262 rows x 8 columns]

In [59]:
rooftop_solar_array_episodes = pd.concat([group.fillna(0) for name, group in rooftop_solar_array_episodes if len(group.dropna()) >= 10000 and group.index[0] >= datetime(2007, 1, 1) and group.index[-1] <= datetime(2008, 12, 31)]).groupby(pd.Grouper(freq='W'))

In [60]:
rooftop_solar_array_episodes.describe()

energy                                                        \
              count      mean       std       min    25%    50%       75%   
Datetime                                                                    
2007-01-07  10080.0  0.435433  1.621301 -0.334581 -0.303 -0.303 -0.124053   
2007-01-14  10080.0  0.527458  1.728628 -0.409348 -0.303 -0.303  0.183083   
2007-01-21  10080.0  0.547682  1.821055 -0.303000 -0.303 -0.303  0.010922   
2007-01-28  10080.0  0.831198  2.209963 -0.303000 -0.303 -0.303  0.644646   
2007-02-04  10080.0  0.604159  1.773673 -0.303000 -0.303 -0.303  0.936483   
...             ...       ...       ...       ...    ...    ...       ...   
2008-11-30  10080.0  0.674482  1.921995 -0.303000 -0.303 -0.303  0.506703   
2008-12-07  10080.0  0.382472  1.454028 -0.348487 -0.303 -0.303  0.186195   
2008-12-14  10080.0  0.479676  1.650902 -0.389125 -0.303 -0.303 -0.142504   
2008-12-21  10080.0  0.274564  1.187142 -0.395298 -0.303 -0.303  0.022552   
2008-12-28  10080.0  0.140639  1.163489 -0.303000 -0.303 -0.303 -0.202468   

                      
                 max  
Datetime              
2007-01-07  6.630260  
2007-01-14  6.792786  
2007-01-21  7.762335  
2007-01-28  9.079978  
2007-02-04  9.024504  
...              ...  
2008-11-30  7.729145  
2008-12-07  6.832454  
2008-12-14  6.691248  
2008-12-21  6.345649  
2008-12-28  6.244976  

[104 rows x 8 columns]

In [61]:
demand =  pd.HDFStore('./minutely/flexible_demand_response.h5')
with pd.HDFStore('./minutely/rooftop_solar_array.h5') as store:
    # Save each DataFrame with a key
    i = 0
    for name, group in rooftop_solar_array_episodes:
        if name.date() == demand[f"eps_{i}"].index[-1].date():
            store[f'eps_{i}'] = group
            i += 1
file = pd.HDFStore('./minutely/rooftop_solar_array.h5')
print(len(file) == len(demand))
file.close()
demand.close()

True
